UI to work with SBI stuff

In [1]:
import os
import subprocess

def source_lmod_script(script_path):
    """
    Source an Lmod/module script and import environment variables into Python safely,
    suppressing terminal warnings.
    """
    # Use a non-interactive login shell (bash -l), redirect errors
    command = f'bash -l -c "source {script_path} >/dev/null 2>&1; printenv -0"'
    
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    out, _ = proc.communicate()
    
    # Parse null-separated environment variables
    for env_var in out.split(b'\0'):
        if env_var:
            key, _, value = env_var.partition(b'=')
            os.environ[key.decode()] = value.decode()

# Example usage
M3_BUILD_DIR = "/home/henryi/scratch/venvs/.venv_sbi/bin/"
TUTORIAL_BUILD_DIR = M3_BUILD_DIR
source_lmod_script(f"{M3_BUILD_DIR}/setup.MaCh3.sh")
source_lmod_script(f"{TUTORIAL_BUILD_DIR}/setup.MaCh3Tutorial.sh")
os.environ["OMP_NUM_THREADS"] = "8"

import logging
import sys

my_stderr = sys.stderr = open('errors.txt', 'w')  # redirect stderr to file
get_ipython().log.handlers[0].stream = my_stderr  # log errors to new stderr
get_ipython().log.setLevel(logging.INFO)  # errors are logged at info level

cat: write error: Broken pipe
cat: write error: Broken pipe


In [2]:
from mach3sbitools.ui.sbi_ui import MaCh3SbiUI
from mach3sbitools.utils.device_handler import TorchDeviceHander
from mach3sbitools.diagnostics.diagnostics import SBIDiagnostics

from tqdm.autonotebook import tqdm
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

Couldn't find DUNE instance
Found MaCh3 Tutorial instance, using that!
FOUND


In [3]:
input_file = Path("/home/henryi/sft/MaCh3Tutorial/TutorialConfigs/FitterConfig.yaml")
ui_main = MaCh3SbiUI(str(input_file), 'tutorial')
device_handler = TorchDeviceHander()

Trying to open /home/henryi/sft/MaCh3Tutorial/TutorialConfigs/FitterConfig.yaml with tutorial
Opening files with Tutorial!
[Monitor.cpp][info] ##################################
[Monitor.cpp][info] Welcome to:  
[Monitor.cpp][info]   __  __        _____ _     ____  
[Monitor.cpp][info]  |  \/  |      / ____| |   |___ \ 
[Monitor.cpp][info]  | \  / | __ _| |    | |__   __) |
[Monitor.cpp][info]  | |\/| |/ _` | |    | '_ \ |__ < 
[Monitor.cpp][info]  | |  | | (_| | |____| | | |___) |
[Monitor.cpp][info]  |_|  |_|\__,_|\_____|_| |_|____/ 
[Monitor.cpp][info] Version: 2.3.1
[Monitor.cpp][info] ##################################
[Monitor.cpp][info] Using following CPU:
[Monitor.cpp][info] model name	: AMD EPYC 9454 48-Core Processor
[Monitor.cpp][info] cpu MHz		: 3799.757
[Monitor.cpp][info] Architecture:                         x86_64
[Monitor.cpp][info] L1d cache:                            1.5 MiB (48 instances)
[Monitor.cpp][info] L1i cache:                            1.5 MiB (48 instan

In [4]:
# Fit settings!
FIT_TYPE='mechanistic_embedding'
N_ROUNDS = 2
RE_TRAINS = 5

In [5]:
SAMPLES_SIZES = [10, 100]#, 1000, 5000, 10000, 50000]
UIS = []
SAMPLES = []

In [6]:
# Now we can run the fit!
for s in tqdm(SAMPLES_SIZES, desc="Training many algorithms"):
    uis_s = []
    samples_s = []
    for r in tqdm(range(RE_TRAINS), desc=f"Training for {s} samples"):
        new_ui = MaCh3SbiUI(str(input_file), 'tutorial')
        new_ui.initialise_fitter(FIT_TYPE, N_ROUNDS, s)
        new_ui.train()
        uis_s.append(new_ui)
        samples_s.append(new_ui.fitter.sample(10_000))

    UIS.append(uis_s)
    SAMPLES.append(SAMPLES)

Training many algorithms:   0%|          | 0/2 [00:00<?, ?it/s]

Training for 10 samples:   0%|          | 0/5 [00:00<?, ?it/s]

Trying to open /home/henryi/sft/MaCh3Tutorial/TutorialConfigs/FitterConfig.yaml with tutorial
Opening files with Tutorial!


TypeError: MaCh3SbiUI.initialise_fitter() missing 1 required positional argument: 'samples_per_round'

In [ ]:
# # Prior check
# prior_samples = ui.fitter.prior.sample((100000,))
# for i, lab in enumerate(ui.mach3.get_parameter_names()):
#     plt.hist(prior_samples[:, i].cpu(), bins=50, density=True)
#     plt.xlabel(lab)
#     plt.title(f"Prior samples for {lab}")
#     plt.show()

In [ ]:
import uproot as ur
tree = ur.open("/home/henryi/sft/MaCh3Tutorial/Test.root:posteriors")
arrs = tree.arrays(library="np")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_parameter_histograms(
    datasets,
    nominal,
    names,
    bins=30,
    burnin=1000,
    alpha=0.7
):
    """
    datasets : list of 2D numpy arrays, shape (n_samples, n_params)
    nominal  : dict-like, nominal[name] -> 1D array
    names    : list of parameter names
    """

    print(datasets[0][0])
    
    for i, name in tqdm(enumerate(names), total=len(names)):
        plt.figure()

        # Extract data for this parameter
        dataset_arrays = [d[:, i].cpu() for d in datasets]
        nominal_array = np.asarray(nominal[name])[burnin:]

        # Combine for binning
        all_arrays = dataset_arrays + [nominal_array]

        # Define bin edges ONCE per parameter
        bin_edges = np.histogram_bin_edges(
            np.concatenate(all_arrays),
            bins=bins
        )

        # Plot dataset histograms (outlines)
        for arr in dataset_arrays:
            plt.hist(
                arr,
                bins=bin_edges,
                density=True,
                histtype="step",
                linewidth=1.0,
                alpha=alpha
            )

        # Plot nominal (black, thicker)
        plt.hist(
            nominal_array,
            bins=bin_edges,
            density=True,
            histtype="step",
            linewidth=2.0,
            color="black",
            label="Nominal"
        )

        plt.xlabel(name)
        plt.ylabel("Posterior Density")
        plt.title(name)
        plt.legend()
        plt.tight_layout()
        plt.show()


In [ ]:
names = ui_main.mach3.get_parameter_names()

plot_parameter_histograms(
    SAMPLES,
    arrs,
    names,
    50,
    10000,
    0.2
)

In [ ]:
# Dumb stuff
from sbi.analysis.plot import sbc_rank_plot
from sbi.diagnostics import check_sbc, check_tarp, run_sbc, run_tarp


num_posterior_samples = 1000
num_sbc_samples = 100  # choose a number of sbc runs, should be ~100s


thetas = ui.fitter.prior.sample(num_sbc_samples)
xs, thetas = ui.fitter.get_x_vals(thetas)


ranks, dap_samples = run_sbc(
    thetas,
    xs,
    ui.fitter.posterior,
    num_posterior_samples=num_posterior_samples,
    num_workers=8,
    use_sample_batched=True,  # True can give a speed-up, but might cause memory issues.
)



In [ ]:
check_stats = check_sbc(
    ranks, thetas, dap_samples, num_posterior_samples=num_posterior_samples
)

print(
    f"""kolmogorov-smirnov p-values \n
    check_stats['ks_pvals'] = {check_stats['ks_pvals'].numpy()}"""
)

print(
    f"c2st accuracies \ncheck_stats['c2st_ranks'] = {check_stats['c2st_ranks'].numpy()}"
)

print(f"- c2st accuracies check_stats['c2st_dap'] = {check_stats['c2st_dap'].numpy()}")




In [ ]:
_ = sbc_rank_plot(
    ranks=ranks,
    num_posterior_samples=num_posterior_samples,
    parameter_labels=ui.mach3.get_parameter_names(),
    plot_type="hist",
    num_bins=None,  # by passing None we use a heuristic for the number of bins.
)


In [ ]:
sbc_rank_plot(ranks, 1_000, plot_type="cdf", parameter_labels=ui.mach3.get_parameter_names())

In [ ]:
ranks, dap_samples = run_sbc(
    thetas,
    xs,
    posterior=ui.fitter.posterior,
    num_posterior_samples=num_posterior_samples,
    reduce_fns= ui.fitter.posterior.log_prob,
)
check_stats = check_sbc(ranks, thetas, dap_samples, 1_000)
print(check_stats)
